# Manual Annotation

# Annotation Guide

You will be required to annotate 42 images by providing answers for three categories: age, gender, and ethnicity. 

### Instructions

#### Please run the code below, and two pop-up windows should appear. 

1. **Enter a Nickname**: Please start by entering a nickname that cannot be linked to your identity. This ensures anonymity in the annotation process.
  
2. **Annotate Each Image**: You will then see the first image. For each image, select one option for each category: **age**, **gender**, and **ethnicity**. Your choices should be based on your best judgment using the definitions provided below:
   - **Age Groups**: Choose from the following:
      - **Baby** (0–1 year)
      - **Child** (2–9 years)
      - **Adolescent** (10–19 years)
      - **Young Adult** (20–29 years)
      - **Middle-aged Adult** (30–49 years)
      - **Older Adult** (50–65 years)
      - **Elderly** (65+ years)
      
   - **Gender**: Choose from **Man** or **Woman**. Note that this category refers to perceived gender based on appearance.
   
   - **Ethnicity**: Choose from the following categories:
      - **Asian**
      - **Indian**
      - **Black**
      - **White**
      - **Middle Eastern**
      - **Latino/Hispanic**
      
   Please base your judgments on visible traits. Keep in mind that these labels are intended for research into automated systems and may not capture the complexity of individual identities.

3. **Save Your Selections**: After selecting an option for all three categories, click on "Save."

4. **Proceed to the Next Image**: Click "Next" to move to the next image.

5. **Repeat the Process**: Continue this process for all images.

6. **Review Completed Annotations**: After completing all annotations, review your entries to ensure that no fields are left blank.

### Important Notes
- **Accuracy and Judgment**: Please annotate each image as accurately as possible, based on your best judgment. If you are uncertain, make your best guess. This helps us assess which images are challenging for annotators, an important part of the analysis.
  
- **Ground Truth and Consistency**: Since this dataset consists of unknown individuals, we do not have exact demographic information. We aim to use your annotations to establish a "ground truth" based on consensus among annotators, which will help us evaluate automated system performance.

- **Ethical Considerations**: Recognize that categories like age, gender, and ethnicity involve subjective assessment and can be sensitive. The purpose of this annotation is to evaluate biases in automated systems and improve their accuracy, not to define individuals. Please approach the task respectfully, keeping in mind the limitations of these categories.

- **Why Your Input Matters**: Your responses are crucial for understanding annotation consistency, identifying challenging cases, and evaluating automated system performance. Your work helps build more accurate and fair models in the future.

In [ ]:
import tkinter as tk
from tkinter import ttk, messagebox, simpledialog
from PIL import Image, ImageTk
import pandas as pd
import os
from functools import reduce
import matplotlib.pyplot as plt

In [ ]:
import os
import tkinter as tk
from tkinter import ttk, messagebox, simpledialog
from PIL import Image, ImageTk
import pandas as pd

class AnnotationTool:
    def __init__(self, root, image_dir, annotations_file):
        self.root = root
        self.image_dir = image_dir
        self.annotations_file = annotations_file

        # Load images from directory and subdirectories
        self.image_files = []
        for subdir, _, files in os.walk(image_dir):
            for file in files:
                if file.endswith(('.jpg', '.png')):
                    # Append full path to the image file
                    self.image_files.append(os.path.join(subdir, file))
        
        self.current_index = 0

        # Load or create annotation DataFrame
        self.annotations_df = self.load_or_create_annotations()

        # Store the current image reference to avoid garbage collection
        self.image_ref = None

        # Build GUI components
        self.build_gui()

        # Defer the nickname prompt until the main window is ready
        self.root.after(100, self.prompt_nickname)

        # Display the first image
        self.display_image()

    def prompt_nickname(self):
        """Prompt the user to enter their nickname for annotations."""
        nickname = simpledialog.askstring("Annotator Name", "Please enter your nickname:")
        if not nickname:
            nickname = "Anonymous"  # Default to Anonymous if no name is provided
        self.annotator = nickname  # Store the nickname for later use

    def load_or_create_annotations(self):
        """Load existing annotations or create a new DataFrame."""
        if os.path.exists(self.annotations_file):
            return pd.read_csv(self.annotations_file)
        else:
            return pd.DataFrame(columns=["Image", "Age", "Gender", "Ethnicity", "Emotion", "Annotation_by"])

    def build_gui(self):
        """Setup the layout of the GUI components."""
        # Image display label
        self.image_label = tk.Label(self.root)
        self.image_label.grid(row=0, column=0, columnspan=2)

        # Age dropdown (Combobox)
        tk.Label(self.root, text="Age:").grid(row=1, column=0)
        self.age_combobox = ttk.Combobox(
            self.root, 
            values=[
                "Baby (0-1)", "Child (2-9)", "Adolescent (10-19)", 
                "Young Adult (20-29)", "Middle-aged Adult (30-49)", 
                "Older Adult (50-65)", "Elderly (65+)"
            ]
        )
        self.age_combobox.grid(row=1, column=1)

        # Gender dropdown (Combobox)
        tk.Label(self.root, text="Gender:").grid(row=2, column=0)
        self.gender_combobox = ttk.Combobox(self.root, values=["Man", "Woman"])
        self.gender_combobox.grid(row=2, column=1)

        # Ethnicity dropdown (Combobox)
        tk.Label(self.root, text="Ethnicity:").grid(row=3, column=0)
        self.ethnicity_combobox = ttk.Combobox(
            self.root, 
            values=["Asian", "Indian", "Black", "White", "Middle Eastern", "Latino/Hispanic"]
        )
        self.ethnicity_combobox.grid(row=3, column=1)

        # Reminder label (placed above Save and Next buttons)
        self.reminder_label = tk.Label(self.root, text="Please click on Save after annotating the image.", fg="blue")
        self.reminder_label.grid(row=4, column=0, columnspan=2)

        # Save button
        self.save_button = tk.Button(self.root, text="Save", command=self.save_annotation)
        self.save_button.grid(row=5, column=0)

        # Next image button
        self.next_button = tk.Button(self.root, text="Next", command=self.next_image)
        self.next_button.grid(row=5, column=1)

    def display_image(self):
        """Load and display the current image, and populate any existing annotations."""
        image_path = self.image_files[self.current_index]

        # Check if image path exists
        if not os.path.exists(image_path):
            messagebox.showerror("Error", f"Image path does not exist: {image_path}")
            return

        img = Image.open(image_path)
        img.thumbnail((400, 400))  # Resize image for display

        # Reassign self.image_ref to prevent garbage collection
        self.image_ref = ImageTk.PhotoImage(img)
        
        # Display the image in the label
        self.image_label.config(image=self.image_ref)

        # Load existing annotations if any
        current_image = os.path.basename(image_path)
        self.load_existing_annotations(current_image)

    def load_existing_annotations(self, current_image):
        """Load existing annotations for the current image if available."""
        if current_image in self.annotations_df['Image'].values:
            annotation = self.annotations_df[self.annotations_df['Image'] == current_image].iloc[0]
            self.age_combobox.set(annotation['Age'])
            self.gender_combobox.set(annotation['Gender'])
            self.ethnicity_combobox.set(annotation['Ethnicity'])
        else:
            # Clear entries if no annotation is found
            self.age_combobox.set('')
            self.gender_combobox.set('')
            self.ethnicity_combobox.set('')

    def get_emotion_from_path(self, image_path):
        """Extract emotion from the folder name in the image path."""
        emotion = os.path.basename(os.path.dirname(image_path))
        return emotion

    def save_annotation(self):
        """Save the current annotation to the DataFrame."""
        current_image = os.path.basename(self.image_files[self.current_index])
        age = self.age_combobox.get()
        gender = self.gender_combobox.get()
        ethnicity = self.ethnicity_combobox.get()
        emotion = self.get_emotion_from_path(self.image_files[self.current_index])

        # Check if all fields are filled in
        if not age or not gender or not ethnicity:
            messagebox.showwarning("Incomplete Data", "Please fill in all fields before saving.")
            return

        # Update or add annotation
        if current_image in self.annotations_df['Image'].values:
            self.annotations_df.loc[self.annotations_df['Image'] == current_image, 
                                    ['Age', 'Gender', 'Ethnicity', 'Emotion', 'Annotation_by']] = [age, gender, ethnicity, emotion, self.annotator]
        else:
            self.annotations_df = self.annotations_df.append(
                {"Image": current_image, "Age": age, "Gender": gender, "Ethnicity": ethnicity, "Emotion": emotion, "Annotation_by": self.annotator},
                ignore_index=True
            )

        # Save annotations to file
        self.annotations_df.to_csv(self.annotations_file, index=False)

        # Show a confirmation popup
        messagebox.showinfo("Saved", f"Annotation for {current_image} saved successfully!")

    def next_image(self):
        """Navigate to the next image."""
        if self.current_index < len(self.image_files) - 1:
            self.current_index += 1
        else:
            self.current_index = 0  # Loop back to the first image
        self.display_image()

# Main section to run the GUI
if __name__ == '__main__':
    # Create the root window
    root = tk.Tk()
    root.title("Image Annotation Tool")

    # Specify the directory where images are located and the CSV file to save annotations
    image_directory = '../../3_image_datasets/affectnet/SelectionManualAnnotation/AffectNet'  # Replace with your directory
    annotations_file = 'annotations_3.csv'  # CSV file for saving annotations

    # Initialize the annotation tool
    app = AnnotationTool(root, image_directory, annotations_file)

    # Start the GUI loop
    root.mainloop()


In [ ]:
annot3_data = pd.read_csv('annotations_3.csv')

In [ ]:
expected_count = 42
actual_count = annot3_data.shape[0]

if actual_count != expected_count:
    print("\n" + "=" * 50)
    print("WARNING: PLEASE CHECK YOUR ANNOTATIONS!")
    print(f"EXPECTED {expected_count} ANNOTATIONS BUT FOUND {actual_count}.")
    print("YOU HAVEN'T SAVED ALL OF THEM.")
    print("=" * 50 + "\n")
else:
    print("All annotations are complete.")
